# PDF RAG Model Testing Notebook
This notebook demonstrates how to use the PDF RAG model to upload a PDF, extract text, chunk, embed, retrieve relevant chunks, and generate answers using OpenAI GPT.

In [ ]:
# Install required packages if not already installed


In [1]:
import os
import tempfile
import traceback
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import numpy as np
import openai

In [2]:
# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

d:\ananconda\envs\globalenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anant\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ''
        for page in reader.pages:
            text += page.extract_text() or ''
        return text
    except Exception as e:
        raise RuntimeError(f'Failed to extract text: {str(e)}')

In [4]:
def chunk_text(text, chunk_size=500):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

In [5]:
def embed_chunks(chunks):
    try:
        embeddings = embedder.encode(chunks)
        return embeddings
    except Exception as e:
        raise RuntimeError(f'Embedding failed: {str(e)}')

In [6]:
def retrieve_relevant_chunks(question, pdf_chunks, pdf_embeddings, top_k=3):
    try:
        question_embedding = embedder.encode([question])[0]
        similarities = np.dot(pdf_embeddings, question_embedding)
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        return [pdf_chunks[i] for i in top_indices]
    except Exception as e:
        raise RuntimeError(f'Retrieval failed: {str(e)}')

In [18]:
import requests
import os

def generate_answer(question, context):
    try:
        api_key = "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"
        if not api_key:
            raise RuntimeError("GROQ_API_KEY environment variable is not set. Please set your Groq API key.")
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": "llama3-8b-8192",  # You can change to llama2-70b-4096 or gemma-7b-it if desired
            "messages": [
                {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"}
            ],
            "max_tokens": 300
        }
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code != 200:
            raise RuntimeError(f"Groq API error: {response.status_code} {response.text}")
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    except Exception as e:
        raise RuntimeError(f"Generation failed: {str(e)}")

## Upload and Process a PDF

In [16]:
# Change 'sample.pdf' to your PDF file path
pdf_path = 'sample.pdf'
text = extract_text_from_pdf(pdf_path)
chunks = chunk_text(text)
embeddings = embed_chunks(chunks)

d:\ananconda\envs\globalenv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## Ask a Question About the PDF

In [21]:
question = input("Enter your question: ")
relevant_chunks = retrieve_relevant_chunks(question, chunks, embeddings)
context = ''.join(relevant_chunks)
answer = generate_answer(question, context)
print('Answer:', answer)

d:\ananconda\envs\globalenv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer: There is no mention of the weather in the text provided. The text appears to be a excerpt from the Constitution of India, and it does not contain any information about the weather.
